In [16]:
import numpy as np
import mat73
from IPython.display import display
import data_utils
import importlib
importlib.reload(data_utils)
from data_utils import nan_sanity_check, calculate_correlation
from plot_functions import reorganize_roi_gradient, reorder_hemispheres
import pickle
import pandas as pd
import scipy
import enigmatoolbox
from enigmatoolbox.utils.parcellation import parcel_to_surface
from enigmatoolbox.plotting import plot_cortical
import mne 
from mne.stats import fdr_correction


In [2]:
data_epi = mat73.loadmat("data//EPI_data.mat")
data_epi = data_epi['EPI_data']

data_HC = mat73.loadmat("data//HC_data.mat")
data_HC = data_HC['HC_data']

epi_labels = [int(value[2]) for value in data_epi]
key_labels = [f"sub{i+1}" for i in range(len(epi_labels))]
epi_labels_dict = dict(zip(key_labels, epi_labels))

del(data_epi); del(data_HC)

In [3]:
### Loading of the already calculated FOOOF exponents dictionaries ###
with open('fooof_data_HC_1_35.pkl', 'rb') as file:
    fooof_dict_HC = pickle.load(file)
with open('fooof_data_EPI_1_35.pkl', 'rb') as file:
    fooof_dict_EPI = pickle.load(file)

### checking for NaN values deriving from unsuccesful FOOOF calculations ###
nan_sanity_check(fooof_dict_EPI, 'EPI 1-35')
nan_sanity_check(fooof_dict_HC, 'HC 1-35')

### Loading of the scores of the epileptic patients in different clinical/neuropsychologica scores in a dataframe ###
measures_df_1_35 = pd.read_csv('data\\neuropsy_scores.csv', sep=';', decimal=",",thousands='.', na_values=['-','--','---'])

No Nan values present in group EPI 1-35
No Nan values present in group HC 1-35


In [11]:
### removing subjects already excluded from the epileptic patients time series ###

for idx in [2,18,45,55,66]:
  measures_df_1_35.drop(measures_df_1_35[measures_df_1_35['Number_of_SJ'] == idx].index, inplace=True)

measures_df_1_35.shape

In [37]:
list_fooof = [list(value) for value in fooof_dict_EPI.values()]
average_fooof_1_35 = np.array(list_fooof).mean(axis=0)

measures_df_1_35['fooof'] = [fooof_dict_EPI[f'sub{sub}'] for sub in measures_df_1_35['Number_of_SJ']]

In [9]:
'''### Subdividing the epileptic subjects group into subdiagnosis groups aand creating a dataframe for each
left_epi_subj = [int(key.replace("sub", "")) for key in fooof_dict_EPI if epi_labels_dict[key]==1]
right_epi_subj = [int(key.replace("sub", "")) for key in fooof_dict_EPI if epi_labels_dict[key]==2]
bilat_epi_subj = [int(key.replace("sub", "")) for key in fooof_dict_EPI if epi_labels_dict[key]==3]

left_epi_measures_df = measures_df_1_35[measures_df_1_35['Number_of_SJ'].isin(left_epi_subj)]
right_epi_measures_df = measures_df_1_35[measures_df_1_35['Number_of_SJ'].isin(right_epi_subj)]
bilat_epi_measures_df = measures_df_1_35[measures_df_1_35['Number_of_SJ'].isin(bilat_epi_subj)]

# adding to each dataframe the fooof values for each subject in tha subdiagnosis group
left_epi_measures_df['fooof'] = [fooof_dict_EPI[f'sub{sub}'] for sub in left_epi_measures_df['Number_of_SJ']]
right_epi_measures_df['fooof'] = [fooof_dict_EPI[f'sub{sub}'] for sub in right_epi_measures_df['Number_of_SJ']]
bilat_epi_measures_df['fooof'] = [fooof_dict_EPI[f'sub{sub}'] for sub in bilat_epi_measures_df['Number_of_SJ']]
'''

C:\Users\simon\AppData\Local\Temp\ipykernel_26244\1151428209.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left_epi_measures_df['fooof'] = [fooof_dict_EPI[f'sub{sub}'] for sub in left_epi_measures_df['Number_of_SJ']]
C:\Users\simon\AppData\Local\Temp\ipykernel_26244\1151428209.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  right_epi_measures_df['fooof'] = [fooof_dict_EPI[f'sub{sub}'] for sub in right_epi_measures_df['Number_of_SJ']]
C:\Users\simon\AppData\Local\Temp\ipykernel_26244\1151428209.p

In [38]:
### Creating a nested dict for every epiulepsy sub-diagnosis 
alpha = 0.05

measures_list = ['ieds','onset','duration','drugs','corsi','ltm_ve_imm','ltm_ve_diff','rey_copy','rey_reproduction','qit','tmta','tmtb']
significance_check = []

corrected_total_correlation_df = pd.DataFrame()

#sub_groups_descriptions = ['All TLE', 'Left TLE', 'Right TLE', 'Bilateral TLE']
columns_list = measures_df_1_35.columns[1:]

#inner loop for the measures of interes
for measure, column in zip(measures_list, columns_list):
  ###  RISOLVERE IL PROBLEMA DEL DROPNA E SIAMO A CAVALLO ###
  dropna_df = measures_df_1_35[['fooof', column]].dropna(subset = [column], inplace = False, axis = 0)
  np.shape(dropna_df)

  corrected_corr, corrected_p_value, corrected_mask = calculate_correlation(dropna_df['fooof'].tolist(), dropna_df[column].tolist(), type='spearman', alpha=alpha, fdr=True) 
  check = 0
  for roi in corrected_mask:
    if roi == 0:
      continue
    elif roi != 0:
      check = 1
      break
  significance_check.append(check)
  corrected_total_correlation_df[measure] = (corrected_corr, corrected_p_value, corrected_mask)

In [39]:
significant_scores = [measures_list[idx] for idx, value in enumerate(significance_check) if value == 1]
print(f'The only clinical/neuropsycological measures that significantly correlates with EPI FOOOF exponent in at least one ROI are {significant_scores}')

The only clinical/neuropsycological measures that significantly correlates with EPI FOOOF exponent in at least one ROI are ['drugs', 'ltm_ve_imm']


In [14]:
'''### Creating a nested dict for every epilepsy sub-diagnosis 
alpha = 0.05

measures_list = ['ieds','onset','duration','drugs','corsi','mlt_ve_imm','mlt_ve_diff','rey_copia','rey_riproduzione','qit','tmta','tmtb']

corrected_total_correlation_df, corrected_left_correlation_df, corrected_right_correlation_df, corrected_bilat_correlation_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

epi_df_corrected_list =  [corrected_total_correlation_df, corrected_left_correlation_df, corrected_right_correlation_df, corrected_bilat_correlation_df]

sub_groups = [measures_df_1_35, left_epi_measures_df, right_epi_measures_df, bilat_epi_measures_df]
#sub_groups_descriptions = ['All TLE', 'Left TLE', 'Right TLE', 'Bilateral TLE']
columns_list = measures_df_1_35.columns[1:]

#outer loop for the subdiagnosis group
for i in range(len(sub_groups)):
  print(f'creating dataframe of group: {sub_groups_descriptions[i]}')
  diagnosis, epi_corrected_df = sub_groups[i], epi_df_corrected_list[i]
  #inner loop for the measures of interes
  for measure, column in zip(measures_list, columns_list):
    ###  RISOLVERE IL PROBLEMA DEL DROPNA E SIAMO A CAVALLO ###
    dropna_df = diagnosis[['fooof', column]].dropna(subset = [column], inplace = False, axis = 0)
    np.shape(dropna_df)

    corrected_corr, corrected_p_value, corrected_mask = calculate_correlation(dropna_df['fooof'].tolist(), dropna_df[column].tolist(), type='spearman', alpha=alpha, fdr=True) 

    epi_corrected_df[measure] = (corrected_corr, corrected_p_value, corrected_mask)
'''


creating dataframe of group: All TLE
creating dataframe of group: Left TLE
creating dataframe of group: Right TLE
creating dataframe of group: Bilateral TLE


In [66]:
'''labels_name = pd.read_excel("Desikan_regions.xlsx").values.tolist()

# Define the column headers
headers = ["Regions", "rho", "p-values"]
rho_1_35_allTLE_table = [round(value, 4) for value in corrected_total_correlation_df['mlt_ve_imm'][0]]
p_1_35_allTLE_table = [round(value, 4) if value >= 0.0001 else "{:.1e}".format(value) for value in corrected_total_correlation_df['mlt_ve_imm'][1]]

data = list(zip(labels_name, rho_1_35_allTLE_table, p_1_35_allTLE_table))
data = [(i[0].replace("'", ""),j,k) for i,j,k in data]
# Create LaTeX table
df = pd.DataFrame(data, columns = headers)
df_filtered = df[df["p-values"].astype('float') <= 0.05]'''

In [35]:
interactive = True
### to correctly plot the gradients obtained from BrainStorm data some indeces transformations are necessary with reorder_emispheres() and reorganize_roi_gradients()

# Map parcellated data to the surface
mat = []
# column value determines wich column of the correlation dataframe (that is, which score) is chosen for the plot
for vec in [corrected_total_correlation_df['drugs'][2], corrected_total_correlation_df['ltm_ve_imm'][2]]:
    corr_mat_plot = reorganize_roi_gradient(reorder_hemispheres(vec))
    corr_mat_plot = (parcel_to_surface(corr_mat_plot, 'aparc_fsa5'))
    mat.append(corr_mat_plot)

# Project the results on the surface brain
if interactive==True:
    # Project the results on the surface brain
    plot_cortical(array_name=mat, surface_name="fsa5", size=(4000, 2800), label_text= {'left':['Drugs','RAVLT imm recall']}, background=(1,1,1), transparent_bg = False, interactive = True, cmap='RdBu_r', color_bar=True, scale=(1,1), zoom = 1.25, color_range=(-0.49, 0.41))
else:
    plot_cortical(array_name=mat, surface_name="fsa5", size=(4000, 2800), screenshot= True, filename=f'IMMAGINI PER PAPER\\Spearman Correlation All TLE fooof vs {column} 1-35 Hz alpha {alpha} FDR corrected.jpeg', background=(1,1,1), transparent_bg = False, interactive = False, cmap='RdBu_r', color_bar=True, scale=(1,1), embed_nb = True, zoom = 1.25, color_range='sym')